In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

**Repository Version** 
This notebook was run at commit:
` `

# Orthogonality Cross-Check
This notebook will compare the results of fits to GlueX-I data in the orthogonal pairs of diamond orientations, as a first check that fits are behaving as expected. 

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import warnings

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1-orthogonal/float-dsratio/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

The files we'll be analyzing are done in the following manner:
* Varying bins of $-t$
  * [0.1, 0.2], [0.2, 0.3], [0.3, 0.5], [0.5, 1.0]
* 20 MeV width bins of mass in the range $1.0 < M_{\omega\pi^0} < 2.0~\text{GeV}$
* Spin $J=1$ amplitudes only, with a common ratio and phase parameter between all $D$ and $S$ waves
* Remove events with $M_{p\pi^0} < 1.4~\text{GeV}$ to remove baryon contributions

In [ ]:
%%bash
# print out the submission files to view the full list of parameters
cat $STUDY_DIR/0-90/submission_0_90.YAML
cat $STUDY_DIR/45-135/submission_45_135.YAML

In [ ]:
from typing import Dict

# Load in the already preprocessed fit results
all_results_0_90 = {}
all_results_45_135 = {}

for bin_dir in sorted(os.listdir(f"{STUDY_DIR}/0-90/")):
    if not os.path.isdir(f"{STUDY_DIR}/0-90/{bin_dir}"):
        continue
    print(f"Loading results from 0-90/{bin_dir}")
    with open(f"{STUDY_DIR}/0-90/{bin_dir}/preprocessed_results.pkl", "rb") as f:
        data = pkl.load(f)
    all_results_0_90[bin_dir] = ResultManager(**data)

for bin_dir in sorted(os.listdir(f"{STUDY_DIR}/45-135/")):
    if not os.path.isdir(f"{STUDY_DIR}/45-135/{bin_dir}"):
        continue
    print(f"Loading results from 45-135/{bin_dir}")
    with open(f"{STUDY_DIR}/45-135/{bin_dir}/preprocessed_results.pkl", "rb") as f:
        data = pkl.load(f)
    all_results_45_135[bin_dir] = ResultManager(**data)

# let interpreter know the types explicitly so we can easily access methods
all_results_0_90: Dict[str, ResultManager]
all_results_45_135: Dict[str, ResultManager]

## Analysis

### Standard Plots
First lets take a look at the standard set of plots to diagnose potential problem areas

In [ ]:
for t_bin_0_90, t_bin_45_135 in zip(sorted(pathlib.Path(f"{STUDY_DIR}/0-90/").iterdir()), sorted(pathlib.Path(f"{STUDY_DIR}/45-135/").iterdir())):
    if not t_bin_0_90.is_dir() or not t_bin_45_135.is_dir():
        continue
    utils.big_print(f"t bin: {t_bin_0_90.name}", 2.0)
    for pdf_0_90 in pathlib.Path(t_bin_0_90 / "plots").glob("*.pdf"):
        for pdf_45_135 in pathlib.Path(t_bin_45_135 / "plots").glob("*.pdf"):            
            if pdf_0_90.name == pdf_45_135.name and pdf_0_90.name in ["jp.pdf", "waves.pdf"]:
                utils.big_print(f"0/90", 1.2)
                utils.display_pdf(str(pdf_0_90), 0)
                utils.big_print(f"45/135", 1.2)
                utils.display_pdf(str(pdf_45_135), 0)

### Background Discontinuity
Around the center of the spectrum, we see a discontinuity pop up where the background and other waves sharply change between two neighboring bins. To understand this, we'll observe the behavior of the 2 bins that surround the discontinuity.

#### Randomized Fits
To start, we'll want to check whether the best of the many randomized fits is a unique solution

In [ ]:
# determine the fit index associated with the neighboring bins in each t bin
t_to_mass_bin = {
    "t_0.1-0.2" : 1.40,
    "t_0.2-0.3" : 1.40,
    "t_0.3-0.5" : 1.42,
    "t_0.5-1.0" : 1.28,
}

t_to_indices = {}

for key, result in all_results_0_90.items():
    mass_bin = t_to_mass_bin[key]
    first_index = result.data_df[result.data_df["m_high"] == mass_bin]["fit_index"].values[0]
    t_to_indices[key] = [first_index + i for i in range(2)]    

In [ ]:
# verify the mass ranges
for t_bin, problem_indices in t_to_indices.items():
    print(
        all_results_0_90[t_bin].data_df[
            all_results_0_90[t_bin].data_df["fit_index"].isin(problem_indices)
        ][["fit_index", "m_low", "m_high"]]
    )

In [ ]:
for t_bin, problem_indices in t_to_indices.items():
    utils.big_print(f"t bin: {t_bin}", 2.0)
    for idx in problem_indices:
        # we know that D-wave errors are not correct right now, and that bootstrap data is not available for the moments
        # so we can suppress these warnings for now
        warnings.filterwarnings("ignore", category=UserWarning) 

        mass_low, mass_high = all_results_0_90[t_bin].data_df[
            all_results_0_90[t_bin].data_df["fit_index"] == idx
        ][["m_low", "m_high"]].values[0]
        
        utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"0/90", 1.2)
        all_results_0_90[t_bin].plot.randomized().randomized_summary(
            idx, likelihood_threshold=10.0, pwa_threshold=0.05, figsize=(15,10)
        )
        plt.show()

        if t_bin == "t_0.3-0.5":
            # for this t bin and 45/135 orientation pair, the discontinuity is in the next bin
            idx += 1  
            mass_low, mass_high = all_results_45_135[t_bin].data_df[
                all_results_45_135[t_bin].data_df["fit_index"] == idx
            ][["m_low", "m_high"]].values[0]
            utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"45/135", 1.2)
        all_results_45_135[t_bin].plot.randomized().randomized_summary(
            idx, likelihood_threshold=10.0, pwa_threshold=0.05, figsize=(15,10)
        )
        plt.show()

warnings.resetwarnings()

#### Angular Distributions
Now lets look at the fits to the angles for any consistent change

In [ ]:
from IPython.display import display, HTML

# use the same indices as above to grab the pdf pages
for t_bin, problem_indices in t_to_indices.items():
    utils.big_print(f"t bin: {t_bin}", 2.0)    

    dir_0_90 = pathlib.Path(f"{STUDY_DIR}/0-90/{t_bin}/plots")
    dir_45_135 = pathlib.Path(f"{STUDY_DIR}/45-135/{t_bin}/plots")
    
    for idx in problem_indices:
        mass_low, mass_high = all_results_0_90[t_bin].data_df[
            all_results_0_90[t_bin].data_df["fit_index"] == idx
        ][["m_low", "m_high"]].values[0]
        
        utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"0/90", 1.2)
        utils.display_pdf(str(dir_0_90 / "combined_angles.pdf"), idx, 200)

        if t_bin == "t_0.3-0.5":
            # for this t bin and 45/135 orientation pair, the discontinuity is in the next bin
            idx += 1  
            mass_low, mass_high = all_results_45_135[t_bin].data_df[
                all_results_45_135[t_bin].data_df["fit_index"] == idx
            ][["m_low", "m_high"]].values[0]
            utils.big_print(f"mass: {mass_low:.2f}-{mass_high:.2f}", 1.5)

        utils.big_print(f"45/135", 1.2)        
        utils.display_pdf(str(dir_45_135 / "combined_angles.pdf"), idx, 200)

Make a plot of the p1p0S, p1ppS, and p1mpP waves across t bins and mass bins to show if they're consistent (strongest waves, and D-wave not interesting due to ratio)

### b1 interference
Lets view how the stable the b1 vector interference is as a function of -t

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,    
    sharey="row",
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(15, 7),
    layout="constrained"
)
masses = result.plot.phase()._masses
bin_width = result.plot.phase()._bin_width
for i, t_bin in enumerate(all_results_0_90.keys()):
    result_0_90 = all_results_0_90[t_bin]
    result_45_135 = all_results_45_135[t_bin]    

    # ##### AMPLITUDES #####

    # --- 0/90 Orientations ---    
    amp1 = axs[0, i].errorbar(
        masses, 
        result_0_90.fit_df["p1p0S"],
        result_0_90.fit_df["p1p0S_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="tab:blue",
        label=rf"{utils.convert_amp_name("p1p0S")} (0/90)",
    )
    amp2 = axs[0, i].errorbar(
        masses, 
        result_0_90.fit_df["p1mpP"],
        result_0_90.fit_df["p1mpP_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="tab:orange",
        label=rf"{utils.convert_amp_name("p1mpP")} (0/90)",
    )

    # --- 45/135 Orientations ---
    amp1_45 = axs[0, i].errorbar(
        masses, 
        result_45_135.fit_df["p1p0S"],
        result_45_135.fit_df["p1p0S_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="tab:blue",
        label=rf"{utils.convert_amp_name("p1p0S")} (45/135)",
    )
    amp2_45 = axs[0, i].errorbar(
        masses, 
        result_45_135.fit_df["p1mpP"],
        result_45_135.fit_df["p1mpP_err"],
        bin_width / 2,
        marker="",
        linestyle="-.",
        markersize=6,
        color="tab:orange",
        label=rf"{utils.convert_amp_name("p1mpP")} (45/135)",
    )

    # Fill between 0_90 and 45_135
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["p1p0S"],
        result_45_135.fit_df["p1p0S"],
        color="tab:blue",
        alpha=0.2,
        step="mid"
    )
    axs[0, i].fill_between(
        masses,
        result_0_90.fit_df["p1mpP"],
        result_45_135.fit_df["p1mpP"],
        color="tab:orange",
        alpha=0.2,
        step="mid"
    )

    # ##### PHASES #####
    # --- 0/90 Orientations ---
    axs[1, i].errorbar(
        masses,
        result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_0_90.fit_df[f"{result_0_90.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        color="black",
    )
    axs[1, i].errorbar(
        masses,
        -result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_0_90.fit_df[f"{result_0_90.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        color="black",
    )    

    # --- 45/135 Orientations ---
    axs[1, i].errorbar(
        masses,
        result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_45_135.fit_df[f"{result_45_135.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        color="black",
    )
    axs[1, i].errorbar(
        masses,
        -result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_45_135.fit_df[f"{result_45_135.phase_difference_dict[('p1p0S', 'p1mpP')]}_err"],
        bin_width / 2,
        marker="",
        linestyle="-",
        color="black",
    )

    # Fill between 0_90 and 45_135
    axs[1, i].fill_between(
        masses,
        result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        color="black",
        linestyle="-.",
        alpha=0.2,
        step="mid"
    )
    axs[1, i].fill_between(
        masses,
        -result_0_90.fit_df[result_0_90.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        -result_45_135.fit_df[result_45_135.phase_difference_dict[("p1p0S", "p1mpP")]].abs(),
        color="black",
        linestyle="-.",
        alpha=0.2,
        step="mid"
    )

    axs[0, i].set_ylim(bottom=0.0)
    low_t, high_t = t_bin.split('_')[1].split('-')
    axs[0, i].set_title(rf" {low_t} < $-t$ <  {high_t} GeV$^2$", pad=10)
    

    axs[1, i].set_yticks(np.linspace(-180, 180, 5))  # force to be in pi intervals
    axs[1, i].set_ylim([-180, 180])    
    axs[1, i].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].set_ylabel(r"Phase Diff. ($^{\circ}$)", loc="center")

# make this a fig legend
axs[0, 3].legend(loc="upper right")

High cos theta baryons appearing around 1.6 or 1.7 GeV
Phi edges begin failing around 1.4 or 1.5